## Empty server

Start a server with no simulations, that just stands as a discoverable hub for clients to connect to:

In [1]:
from nanover.omni import OmniRunner

empty_runner = OmniRunner.with_basic_server()
empty_runner.print_basic_info()

Serving "ragzo: NanoVer iMD Server" (ws://localhost:38801), discoverable on all interfaces on port 54545
Available simulations:



## Client connected to empty server

Connect a client to the empty server:

In [2]:
from nanover.websocket import NanoverImdClient

client = NanoverImdClient.from_runner(empty_runner)

## Simulation

Setup a simulation of interest:

In [3]:
from nanover.openmm import OpenMMSimulation

input_xml = "openmm_files/nanotube.xml"
nanotube_simulation = OpenMMSimulation.from_xml_path(input_xml)
nanotube_simulation.load()

## Simulate from client

Setup a simulation runner that publishes frames via the existing client:

In [4]:
# NBVAL_SKIP
from nanover.omni import OmniRunner

client_runner = OmniRunner.from_client(client)
client_runner.add_simulation(nanotube_simulation)

client_runner.load(0)

## Check with NGLView

Connect an independent NGLClient to the server to check that the simulation is available:

In [5]:
#NBVAL_SKIP
from nanover.jupyter import NGLClient

observer = NGLClient.from_runner(empty_runner)
observer.view

NGLWidget()